In [1]:
import os
import numpy as np
import pandas as pd

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# base directory
BASE_DIR = '..'

# data directory
DATA_DIR = os.path.join(BASE_DIR, 'data')

# raw, interim & processed data directory
RAW_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DIR = os.path.join(DATA_DIR, 'interim')
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')


# raw data
TRAIN = os.path.join(RAW_DIR, 'train')
TEST = os.path.join(RAW_DIR, 'test')
DF_TRAIN = os.path.join(TRAIN, 'train.csv')
SAMPLE_SUBMISSION = os.path.join(RAW_DIR, 'sample_submission.csv')

# interim data
TRAIN_BY_CLASS = os.path.join(INTERIM_DIR, 'train')
VALIDATION_BY_CLASS = os.path.join(INTERIM_DIR, 'validation')
TEST_BY_CLASS = os.path.join(INTERIM_DIR, 'test')

## Datagen

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_BY_CLASS,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

train_generator_ = test_datagen.flow_from_directory(
    TRAIN_BY_CLASS,
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    VALIDATION_BY_CLASS,
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    TEST_BY_CLASS,
    target_size=(150, 150),
    batch_size=1,
    shuffle=False,
    class_mode='categorical')

Found 12578 images belonging to 7 classes.
Found 12578 images belonging to 7 classes.
Found 1475 images belonging to 7 classes.
Found 1475 images belonging to 7 classes.


## Prediction

In [8]:
model = load_model('models/221020/resnet50v2/1/model.10-0.19.h5')

In [10]:
pred = model.predict_generator(test_generator)

InternalError:  cuDNN launch failure : input shape([1,3,156,156]) filter shape([7,7,3,64])
	 [[node sequential/resnet50v2/conv1_conv/Conv2D (defined at <ipython-input-9-f79076686fd5>:1) ]] [Op:__inference_predict_function_5474]

Function call stack:
predict_function


In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames = test_generator.filenames
filenames = [i.split('\\')[0] for i in filenames]

In [ ]:
print(classification_report(filenames, predictions))

In [ ]:
print(confusion_matrix(filenames, predictions))